### Normal Chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)
template = PromptTemplate.from_template("You are a helpful assistant. Answer the question: {question}?")
parser = StrOutputParser()

chain = template | llm | parser

result = chain.invoke({"question": "What is the capital of France?"})
print(result)